<a href="https://colab.research.google.com/github/SATHYARANGARAJ53/medicine_inventory/blob/main/projectmodel_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load dataset
df = pd.read_csv("/content/Medicine_Details.csv", usecols=lambda col: not col.startswith("Unnamed"))  # Replace with actual file path

# Step 1: Handle Missing Values
df.dropna(inplace=True)

# Step 2: Convert Categorical Data
label_encoders = {}
categorical_columns = ["Clinic_ID", "Medicine_Name", "Season"]
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Store encoders for later use

# Step 3: Normalize Consumption & Weather Data
scaler = MinMaxScaler()
numerical_columns = (
    [f"Week_{i}_Consumption" for i in range(1, 17)] +  # Weekly consumption columns
    [f"Week_{i}_Min_Weather" for i in range(1, 17)] +  # Weekly min weather
    [f"Week_{i}_Max_Weather" for i in range(1, 17)]    # Weekly max weather
)
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Step 4: Separate Training Data (2000-2024) and Prediction Data (2025)
df_train = df[df["Year"] < 2025]
df_2025 = df[df["Year"] == 2024]  # Use 2024's last 16 weeks as input for 2025 prediction

# Step 5: Prepare Data for LSTM
sequence_length = 16  # 16 weeks history
data = []
target = []
for i in range(len(df_train)):
    data.append(df_train.iloc[i, 4:20].values)  # Extract 16-week consumption
    target.append(df_train.iloc[i, 4:20].values)  # Predict next 16-week consumption

data = np.array(data).reshape(-1, sequence_length, 1)  # Reshape for LSTM
target = np.array(target)  # Convert target to NumPy array

# Prepare 2025 Prediction Input
data_2025 = df_2025.iloc[:, 4:20].values.reshape(-1, sequence_length, 1)

# Step 6: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42, shuffle=True)

# Step 7: Build LSTM Model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(sequence_length, 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(16)  # Predict next 16 weeks
])

model.compile(optimizer='adam', loss='mse')

# Step 8: Train Model with Accuracy Monitoring
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Step 9: Predict for Test Data
y_pred = model.predict(X_test)

# Convert to NumPy array if necessary
y_test = np.array(y_test)
y_pred = np.array(y_pred)

# Inverse transform predictions and actual values
y_test_actual = scaler.inverse_transform(
    np.hstack([y_test, np.zeros((y_test.shape[0], len(numerical_columns) - 16))])
)[:, :16]
y_pred_actual = scaler.inverse_transform(
    np.hstack([y_pred, np.zeros((y_pred.shape[0], len(numerical_columns) - 16))])
)[:, :16]

# Step 10: Calculate Accuracy Metrics
mae = mean_absolute_error(y_test_actual, y_pred_actual)
mse = mean_squared_error(y_test_actual, y_pred_actual)
rmse = np.sqrt(mse)

print(f"Model Accuracy Metrics:\nMAE: {mae}\nMSE: {mse}\nRMSE: {rmse}")

# Step 11: Predict for 2025
predictions_2025 = model.predict(data_2025)

# Inverse transform to get actual values
predictions_2025 = scaler.inverse_transform(
    np.hstack([predictions_2025, np.zeros((predictions_2025.shape[0], len(numerical_columns) - 16))])
)[:, :16]

# Decode Medicine_Name and Season back to original labels
medicine_names = label_encoders["Medicine_Name"].inverse_transform(df_2025["Medicine_Name"].values)
seasons = label_encoders["Season"].inverse_transform(df_2025["Season"].values)


# Create DataFrame with predictions
predictions_df = pd.DataFrame(predictions_2025, columns=[f"Week_{i+1}_Predicted" for i in range(16)])

# Add Medicine_Name and Season to predictions
predictions_df.insert(0, "Medicine_Name", medicine_names)
predictions_df.insert(1, "Season", seasons)

# Save to CSV
predictions_df.to_csv("/content/predictions_2025.csv", index=False)

print("LSTM Model Training Complete! Predictions saved to predictions_2025.csv with Medicine_Name and Season")



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 268ms/step - loss: 0.4323 - val_loss: 0.1759
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.3709 - val_loss: 0.1470
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.3106 - val_loss: 0.1090
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.2463 - val_loss: 0.0664
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.1387 - val_loss: 0.0542
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.1097 - val_loss: 0.0324
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0711 - val_loss: 0.0145
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0548 - val_loss: 0.0100
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0455 - val_loss: 0.0071
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0538 - val_loss: 0.0053
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0307 - val_loss: 0.0045
Epoch 12/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0371 - val_loss: 0.0039
